# To Yao



In [2]:
import sys
sys.path.append('I:/YaoTony/code/x8313')
# sys.path.append('d:/Codes/EquineTrading/')
from datetime import date
import pytest
from pandas import date_range
import pandas as pd
from horse.betsim.wrap.jcapper import JCapper
from horse.betsim.math import compute_probs_from_odds
import numpy as np

In [17]:
df_payoutpool = pd.read_csv("df_calibrate_payoutpool_2080630.csv")
df_CALIBRATE = pd.read_csv("DF_CALIBRATE_20180630.csv")

## Data Comparison

The only difference between two dataset is that df_calibrate_payoutpool_2080630.csv has 19 more races. And moreover, it has more reduntant data.

In [18]:
df_payoutpool.shape

(4226, 42)

In [44]:
df_CALIBRATE.shape

(1383, 6)

In [40]:
df_CALIBRATE = df_CALIBRATE.sort_values(by=["race_id"])


In [41]:
len(set(df_CALIBRATE["race_id"]))

182

In [39]:
len(set(df_payoutpool["race_id"]))

201

In [49]:
# The races no in df_CALIBRATE
for race_id in set(df_payoutpool["race_id"]):
    if race_id not in set(df_CALIBRATE["race_id"]):
        print(race_id)

ABT_20160703_1
ARP_20160703_1
ABT_20160703_4
LBG_20160703_4
CPW_20160703_8
ABT_20160703_2
GRP_20160703_5
ABT_20160703_7
LRL_20160703_2
ABT_20160703_6
CPW_20160703_9
MNR_20160703_1
ELP_20160703_3
RUI_20160703_7
ELP_20160703_9
ABT_20160703_5
ABT_20160703_3
GRP_20160703_3
CBY_20160703_3


In [54]:
df_payoutpool = df_payoutpool[["race_id","bet_type","payout","pool_name","pool_size","pool_type","bet_amount",\
               "runners","num_starters","pct_cover_exacta","pct_cover_trifecta","pct_cover_superfecta"
]]

## Data Combination
* Question 1: What is pool_type means in df_payoutpool. It shows that even for the same race and bet, there can be different pool_type, so it should not be a feature for race.

* Question 2: We don't have odds information in both dataset 


Note that pool size is a character of pool which denoted as pool_name, WPS means the combined pool of win, place, and show.

In [55]:
# An example process, here only print out the group information.
for race_id in set(df_CALIBRATE["race_id"]):
    print(df_payoutpool[df_payoutpool["race_id"]==race_id])
    print(df_CALIBRATE[df_CALIBRATE["race_id"]==race_id])
    break

             race_id bet_type  payout pool_name  pool_size  pool_type  \
1517  FEX_20160703_4       TR   95.30        TR    18536.0          4   
1518  FEX_20160703_4       EX   27.45        EX    20117.0          3   
1519  FEX_20160703_4       SU  601.75        SU    13144.0          5   
1520  FEX_20160703_4       WN   10.95       WPS    31236.0          0   
1521  FEX_20160703_4       WN   10.95       WPS    31236.0          1   
1522  FEX_20160703_4       WN   10.95       WPS    31236.0          2   
1523  FEX_20160703_4       PL    4.25       WPS    31236.0          0   
1524  FEX_20160703_4       PL    4.25       WPS    31236.0          1   
1525  FEX_20160703_4       PL    4.25       WPS    31236.0          2   
1526  FEX_20160703_4       SH    2.45       WPS    31236.0          0   
1527  FEX_20160703_4       SH    2.45       WPS    31236.0          1   
1528  FEX_20160703_4       SH    2.45       WPS    31236.0          2   
1529  FEX_20160703_4       PL    1.80       WPS    

## Strategy Classification

In [ ]:
def compute_payout(df, attr_model, bet_amount_equal, bet_amount_inequal, bet_on='final_tote_odds'):
    """
    Add columns for quick calculation of Win bets % payout
    :param df: Dataframe from dataset for multiple races
    :param attr_model: (string)an attribute / prob or score in the dataframe that can be ranked
    :param bet_amount_equal: bet_amount when ranking 1 runner according to attr_model is the same with favourate runner according to final tote odds
    :param bet_amount_inequal: bet_amount when ranking 1 runner according to attr_model is different with favourate runner according to final tote odds
    :param bet_on: if bet on ranking 1 runner according to attr_model or favourate runner according to final tote odds
    :return: Dataframe with columns added

    """
    df['is_win'] = df['official_finish_position'].map(lambda x: int(x == 1))
    df['rank_' + attr_model] = df.groupby('race_id')[attr_model].transform(lambda x: x.rank(ascending=False))
    if bet_amount_equal == 'strat_double':
        df['bet_amount'] = df['rank_' + bet_on].map(lambda x: int(x < 1.5) * 2.0)
    elif bet_amount_equal == 'strat_pass':
        df['bet_amount'] = df['rank_' + bet_on].map(lambda x: int(x < 1.5) * 0.0)
    elif bet_amount_equal == 'strat_unchanged':
        df['bet_amount'] = df['rank_' + bet_on].map(lambda x: int(x < 1.5) * 1.0)
    elif bet_amount_equal == 'strat_inverse_scaled':
        df['bet_amount'] = df['rank_' + bet_on].map(lambda x: int(x < 1.5) * 1.0) / df[attr_model]
    else:
        print('bet_amount_equal error!')
        return (None)

    symbol = ['final_tote_odds', attr_model]
    symbol.remove(bet_on)
    symbol_left = symbol[0]

    if bet_amount_inequal == 'strat_double':
        df.loc[(df['rank_' + bet_on] < 1.5) & (df['rank_' + symbol_left] > 1.5), 'bet_amount'] = 2.0
    elif bet_amount_inequal == 'strat_pass':
        df.loc[(df['rank_' + bet_on] < 1.5) & (df['rank_' + symbol_left] > 1.5), 'bet_amount'] = 0.0
    elif bet_amount_inequal == 'strat_unchanged':
        df.loc[(df['rank_' + bet_on] < 1.5) & (df['rank_' + symbol_left] > 1.5), 'bet_amount'] = 1.0
    elif bet_amount_inequal == 'strat_inverse_scaled':
        df.loc[(df['rank_' + bet_on] < 1.5) & (df['rank_' + symbol_left] > 1.5), 'bet_amount'] = 1 / df.loc[
            (df['rank_' + bet_on] < 1.5) & (df['rank_' + symbol_left] > 1.5), attr_model]
    else:
        print('bet_amount_inequal error!')
        return (None)

    df['is_wager'] = df['bet_amount'].map(lambda x: int(x > 0))
    df['is_paid'] = df['is_wager'] * df['is_win']
    df['payout'] = df['is_win'] * df['bet_amount'] * df['payout_win'].fillna(0.0)

    return df